# 2026 MCM Problem C: Dancing with the Stars 数据分析

## 目录
1. 数据加载与预处理
2. 探索性数据分析（EDA）
3. 粉丝投票估算模型
4. 投票机制对比分析
5. 争议案例深度分析
6. 结论与建议

In [ ]:
# 导入必要的库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 设置绘图风格
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# 数据路径
DATA_PATH = r"d:\Data\xwechat_files\wxid_m7pucc5xg5m522_f378\msg\file\2026-01\中文版赛题 (1)\中文版赛题\2026_MCM_Problem_C_Data.csv"

print("库加载完成！")

## 1. 数据加载与预处理

In [ ]:
# 加载原始数据
df_raw = pd.read_csv(DATA_PATH)
print(f"数据集大小: {df_raw.shape}")
print(f"\n列名列表:")
for i, col in enumerate(df_raw.columns):
    print(f"  {i+1}. {col}")

In [ ]:
# 查看前几行数据
df_raw.head(10)

In [ ]:
# 数据预处理函数
def preprocess_data(df):
    """数据预处理"""
    df = df.copy()
    
    # 替换 N/A 为 NaN
    df = df.replace('N/A', np.nan)
    
    # 计算每周评委总分
    for week in range(1, 12):
        judge_cols = [f'week{week}_judge{j}_score' for j in range(1, 5)]
        existing_cols = [col for col in judge_cols if col in df.columns]
        
        if existing_cols:
            for col in existing_cols:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            
            df[f'week{week}_total'] = df[existing_cols].sum(axis=1, skipna=True)
            df[f'week{week}_avg'] = df[existing_cols].mean(axis=1, skipna=True)
    
    # 提取淘汰周数
    def get_elim_week(result):
        if pd.isna(result):
            return -1
        result = str(result).lower()
        if 'week' in result:
            try:
                num = ''.join(filter(str.isdigit, result.split('week')[1]))
                return int(num) if num else -1
            except:
                return -1
        return -1
    
    df['elimination_week'] = df['results'].apply(get_elim_week)
    
    return df

# 预处理数据
df = preprocess_data(df_raw)
print(f"预处理完成！数据集大小: {df.shape}")

## 2. 探索性数据分析 (EDA)

In [ ]:
# 基本统计信息
print("=" * 50)
print("数据集基本信息")
print("=" * 50)
print(f"总选手数: {len(df)}")
print(f"总赛季数: {df['season'].nunique()} (第{df['season'].min()}季 - 第{df['season'].max()}季)")
print(f"选手年龄范围: {df['celebrity_age_during_season'].min():.0f} - {df['celebrity_age_during_season'].max():.0f}")
print(f"选手平均年龄: {df['celebrity_age_during_season'].mean():.1f}")

In [ ]:
# 行业分布
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 饼图
industry_counts = df['celebrity_industry'].value_counts()
axes[0].pie(industry_counts.values, labels=industry_counts.index, autopct='%1.1f%%',
            startangle=90, colors=plt.cm.Set3(np.linspace(0, 1, len(industry_counts))))
axes[0].set_title('Celebrity Industry Distribution', fontsize=14)

# 条形图
industry_counts.plot(kind='barh', ax=axes[1], color='steelblue', alpha=0.7)
axes[1].set_xlabel('Count')
axes[1].set_title('Celebrity Industry Distribution', fontsize=14)

plt.tight_layout()
plt.savefig('figures/industry_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 舞伴成功率分析
partner_stats = df.groupby('ballroom_partner').agg({
    'placement': ['mean', 'count', 'min', 'std']
}).reset_index()
partner_stats.columns = ['partner', 'avg_placement', 'appearances', 'best_placement', 'std_placement']

# 只显示出场次数>=3的舞伴
partner_stats = partner_stats[partner_stats['appearances'] >= 3].sort_values('avg_placement')

print("专业舞伴成功率排名 (出场>=3次):")
print("=" * 60)
for _, row in partner_stats.head(15).iterrows():
    champion = "🏆" if row['best_placement'] == 1 else "  "
    print(f"{champion} {row['partner']:25s} | 平均排名: {row['avg_placement']:.1f} | 出场: {row['appearances']:.0f}次")

In [ ]:
# 绘制舞伴成功率
fig, ax = plt.subplots(figsize=(12, 10))

colors = ['gold' if p == 1 else 'steelblue' for p in partner_stats['best_placement']]
bars = ax.barh(partner_stats['partner'], partner_stats['avg_placement'], 
               color=colors, alpha=0.7, edgecolor='black')

ax.set_xlabel('Average Final Placement (lower is better)', fontsize=12)
ax.set_ylabel('Professional Partner', fontsize=12)
ax.set_title('Professional Partner Success Rate\n(Gold = Won Championship)', fontsize=14)
ax.invert_xaxis()

# 添加数值标签
for bar, (_, row) in zip(bars, partner_stats.iterrows()):
    ax.text(bar.get_width() - 0.3, bar.get_y() + bar.get_height()/2,
            f'{row["avg_placement"]:.1f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('figures/partner_success.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 年龄与最终排名的关系
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 散点图
ax1 = axes[0]
ax1.scatter(df['celebrity_age_during_season'], df['placement'], alpha=0.5, c='steelblue')
ax1.set_xlabel('Age During Season')
ax1.set_ylabel('Final Placement')
ax1.set_title('Age vs Final Placement')

# 添加趋势线
z = np.polyfit(df['celebrity_age_during_season'].dropna(), 
               df.loc[df['celebrity_age_during_season'].notna(), 'placement'], 1)
p = np.poly1d(z)
x_line = np.linspace(df['celebrity_age_during_season'].min(), df['celebrity_age_during_season'].max(), 100)
ax1.plot(x_line, p(x_line), 'r--', label=f'Trend (slope={z[0]:.3f})')
ax1.legend()

# 箱线图：按年龄段
ax2 = axes[1]
df['age_group'] = pd.cut(df['celebrity_age_during_season'], 
                         bins=[0, 25, 35, 45, 55, 100], 
                         labels=['<25', '25-35', '35-45', '45-55', '55+'])
df.boxplot(column='placement', by='age_group', ax=ax2)
ax2.set_xlabel('Age Group')
ax2.set_ylabel('Final Placement')
ax2.set_title('Placement Distribution by Age Group')
plt.suptitle('')

plt.tight_layout()
plt.savefig('figures/age_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. 粉丝投票估算模型

In [ ]:
def get_week_data(df, season, week):
    """获取指定赛季、指定周的在场选手数据"""
    season_df = df[df['season'] == season].copy()
    
    # 筛选该周仍在场的选手
    active_mask = (
        (season_df['elimination_week'] == -1) |  # 进入决赛
        (season_df['elimination_week'] >= week)   # 还未被淘汰
    )
    
    score_col = f'week{week}_total'
    if score_col in season_df.columns:
        active_mask &= (season_df[score_col] > 0)
    
    return season_df[active_mask].copy()

def get_eliminated(df, season, week):
    """获取被淘汰的选手"""
    season_df = df[df['season'] == season]
    eliminated = season_df[season_df['elimination_week'] == week]
    if len(eliminated) > 0:
        return eliminated.iloc[0]['celebrity_name']
    return None

In [ ]:
def estimate_fan_votes_percentage(week_data, eliminated, week):
    """
    百分比制下估算粉丝投票
    
    核心思想：
    - 综合百分比 = 评委得分百分比 + 粉丝投票百分比
    - 被淘汰者的综合百分比最低
    - 通过逆向推理，构造满足约束的粉丝投票
    """
    n = len(week_data)
    score_col = f'week{week}_total'
    
    judge_scores = week_data[score_col].values
    total_judge = judge_scores.sum()
    
    if total_judge == 0:
        return {row['celebrity_name']: 0 for _, row in week_data.iterrows()}
    
    judge_pct = judge_scores / total_judge
    names = week_data['celebrity_name'].values
    
    elim_idx = np.where(names == eliminated)[0]
    if len(elim_idx) == 0:
        return {name: 1.0 / n for name in names}
    elim_idx = elim_idx[0]
    
    # 构造满足约束的粉丝投票
    fan_votes = np.zeros(n)
    base_votes = 1_000_000
    
    for i in range(n):
        if i == elim_idx:
            fan_votes[i] = base_votes * 0.5
        else:
            fan_votes[i] = base_votes * (1.0 + (judge_pct[i] - judge_pct[elim_idx]))
    
    fan_votes = fan_votes / fan_votes.sum() * base_votes * n
    
    return {names[i]: fan_votes[i] for i in range(n)}

def estimate_fan_votes_ranking(week_data, eliminated, week):
    """
    排名制下估算粉丝投票
    
    核心思想：
    - 评委排名 + 粉丝排名 = 综合排名
    - 综合排名最大者淘汰
    """
    n = len(week_data)
    score_col = f'week{week}_total'
    
    judge_scores = week_data[score_col].values
    names = week_data['celebrity_name'].values
    
    # 评委排名（得分越高排名越靠前）
    judge_rank = n - np.argsort(np.argsort(judge_scores))
    
    elim_idx = np.where(names == eliminated)[0]
    if len(elim_idx) == 0:
        return {name: 1.0 / n for name in names}
    elim_idx = elim_idx[0]
    
    fan_votes = np.zeros(n)
    base_votes = 1_000_000
    
    for i in range(n):
        if i == elim_idx:
            fan_votes[i] = base_votes * 0.3
        else:
            fan_votes[i] = base_votes * (1.2 - 0.1 * judge_rank[i])
    
    return {names[i]: fan_votes[i] for i in range(n)}

In [ ]:
def estimate_season_votes(df, season, method='percentage'):
    """估算一个赛季所有周的粉丝投票"""
    season_df = df[df['season'] == season]
    results = {}
    
    # 确定赛季最大周数
    max_week = 11
    for week in range(11, 0, -1):
        col = f'week{week}_total'
        if col in season_df.columns and (season_df[col] > 0).any():
            max_week = week
            break
    
    for week in range(1, max_week + 1):
        week_data = get_week_data(df, season, week)
        eliminated = get_eliminated(df, season, week)
        
        if len(week_data) < 2 or eliminated is None:
            continue
        
        if method == 'percentage':
            votes = estimate_fan_votes_percentage(week_data, eliminated, week)
        else:
            votes = estimate_fan_votes_ranking(week_data, eliminated, week)
        
        results[week] = votes
    
    return results

In [ ]:
# 测试：估算第2季的粉丝投票（Jerry Rice争议赛季）
season_2_votes = estimate_season_votes(df, 2, 'percentage')

print("第2季粉丝投票估算结果:")
print("=" * 60)
for week, votes in season_2_votes.items():
    print(f"\n第{week}周:")
    sorted_votes = sorted(votes.items(), key=lambda x: x[1], reverse=True)
    for name, vote in sorted_votes:
        print(f"  {name:25s}: {vote:,.0f} 票")

In [ ]:
# 可视化第2季的估算投票
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, (week, votes) in enumerate(list(season_2_votes.items())[:6]):
    ax = axes[idx]
    
    names = list(votes.keys())
    values = list(votes.values())
    
    # 截断长名字
    short_names = [n[:12] + '...' if len(n) > 12 else n for n in names]
    
    # 按票数排序
    sorted_idx = np.argsort(values)[::-1]
    short_names = [short_names[i] for i in sorted_idx]
    values = [values[i] for i in sorted_idx]
    
    colors = ['coral' if v == min(values) else 'steelblue' for v in values]
    
    ax.barh(short_names, values, color=colors, alpha=0.7)
    ax.set_xlabel('Estimated Fan Votes')
    ax.set_title(f'Week {week}')
    ax.ticklabel_format(style='scientific', axis='x', scilimits=(0,0))

plt.suptitle('Season 2: Estimated Fan Votes by Week\n(Red = Lowest/Eliminated)', fontsize=14)
plt.tight_layout()
plt.savefig('figures/season2_votes.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. 投票机制对比分析

In [ ]:
def compare_mechanisms(df, season):
    """比较两种投票机制"""
    votes_pct = estimate_season_votes(df, season, 'percentage')
    votes_rank = estimate_season_votes(df, season, 'ranking')
    
    results = []
    season_df = df[df['season'] == season]
    
    for week in votes_pct.keys():
        actual_eliminated = get_eliminated(df, season, week)
        if actual_eliminated is None:
            continue
        
        week_data = get_week_data(df, season, week)
        score_col = f'week{week}_total'
        
        # 计算评委评分排名
        judge_scores = week_data.set_index('celebrity_name')[score_col]
        elim_judge_rank = (judge_scores < judge_scores[actual_eliminated]).sum() + 1
        
        results.append({
            'week': week,
            'eliminated': actual_eliminated,
            'judge_rank': elim_judge_rank,
            'total_contestants': len(week_data),
            'was_lowest_judge': elim_judge_rank == len(week_data)
        })
    
    return pd.DataFrame(results)

# 分析第2季
season_2_comparison = compare_mechanisms(df, 2)
print("第2季淘汰分析:")
season_2_comparison

## 5. 争议案例深度分析

In [ ]:
# 定义争议案例
controversial_cases = [
    {'season': 2, 'name': 'Jerry Rice', 'description': '评委垫底5周仍获亚军'},
    {'season': 4, 'name': 'Billy Ray Cyrus', 'description': '评委垫底6周仍获第5'},
    {'season': 11, 'name': 'Bristol Palin', 'description': '12次评委最低仍获第3'},
    {'season': 27, 'name': 'Bobby Bones', 'description': '评委一直最低仍夺冠'},
]

def analyze_controversial_case(df, season, name):
    """分析争议选手的每周评委排名"""
    season_df = df[df['season'] == season]
    contestant = season_df[season_df['celebrity_name'] == name]
    
    if len(contestant) == 0:
        return None
    
    contestant = contestant.iloc[0]
    
    weekly_data = []
    
    for week in range(1, 12):
        week_data = get_week_data(df, season, week)
        score_col = f'week{week}_total'
        
        if score_col not in week_data.columns or name not in week_data['celebrity_name'].values:
            break
        
        scores = week_data[score_col].values
        names = week_data['celebrity_name'].values
        n = len(names)
        
        # 计算排名
        sorted_idx = np.argsort(scores)[::-1]
        rank_idx = np.where(names[sorted_idx] == name)[0]
        if len(rank_idx) > 0:
            rank = rank_idx[0] + 1
            weekly_data.append({
                'week': week,
                'score': contestant[score_col],
                'rank': rank,
                'total_contestants': n,
                'is_last': rank == n
            })
    
    return pd.DataFrame(weekly_data)

In [ ]:
# 分析所有争议案例
print("争议案例深度分析")
print("=" * 70)

for case in controversial_cases:
    season = case['season']
    name = case['name']
    
    print(f"\n{'='*70}")
    print(f"Season {season}: {name}")
    print(f"描述: {case['description']}")
    print(f"{'='*70}")
    
    # 检查选手是否在数据中
    contestant = df[(df['season'] == season) & (df['celebrity_name'] == name)]
    if len(contestant) == 0:
        print(f"  [注意] 在数据中未找到该选手")
        continue
    
    contestant = contestant.iloc[0]
    print(f"  最终排名: {contestant['placement']}")
    print(f"  舞伴: {contestant['ballroom_partner']}")
    
    case_analysis = analyze_controversial_case(df, season, name)
    if case_analysis is not None and len(case_analysis) > 0:
        print(f"  参赛周数: {len(case_analysis)}")
        print(f"  评委垫底次数: {case_analysis['is_last'].sum()}")
        print(f"  平均评委排名: {case_analysis['rank'].mean():.1f}")
        print(f"\n  每周详情:")
        for _, row in case_analysis.iterrows():
            flag = "⚠️" if row['is_last'] else "  "
            print(f"    {flag} Week {row['week']}: 得分={row['score']:.1f}, "
                  f"排名={row['rank']}/{row['total_contestants']}")

In [ ]:
# 可视化争议案例
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, case in enumerate(controversial_cases):
    ax = axes[idx]
    season = case['season']
    name = case['name']
    
    case_analysis = analyze_controversial_case(df, season, name)
    
    if case_analysis is not None and len(case_analysis) > 0:
        weeks = case_analysis['week'].values
        ranks = case_analysis['rank'].values
        totals = case_analysis['total_contestants'].values
        
        # 绘制排名（反转，让1在上面）
        ax.plot(weeks, ranks, 'o-', color='red', linewidth=2, markersize=8, label=name)
        ax.plot(weeks, totals, 's--', color='gray', alpha=0.5, label='Last Place')
        
        ax.set_xlabel('Week')
        ax.set_ylabel('Judge Rank (1=Best)')
        ax.set_title(f'Season {season}: {name}\n{case["description"]}')
        ax.invert_yaxis()
        ax.legend()
        ax.set_xticks(weeks)
    else:
        ax.text(0.5, 0.5, f'Season {season}: {name}\n数据未找到', 
                ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'Season {season}: {name}')

plt.tight_layout()
plt.savefig('figures/controversial_cases.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. 结论与建议

### 主要发现

1. **粉丝投票的巨大影响**：争议案例清楚地表明，粉丝投票可以完全逆转评委的评判。

2. **两种机制的差异**：
   - 百分比制更容易让高人气选手存活
   - 排名制在选手数量较少时差异更明显

3. **建议的新机制**：
   - 可以考虑设置评委评分门槛
   - 或者动态调整权重（前期重技术，后期重人气）

In [ ]:
# 创建figures目录
import os
os.makedirs('figures', exist_ok=True)
print("分析完成！图表已保存到 figures/ 目录。")